# Using Neural Networks for Subgrid Forcing Parameterizations

Now that we have generated coarsened, low-resolution datasets, we can now feed these datasets as training data for our parameterized machine learning (ML) training sessions. In this section, we will focus on one category of parameterizations, fully convolutional neural networks (FCNNs), though will explore other strata of parameterizations in subsequent sections. 

In [6]:
# Import required modules
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pyqg
import json
import fsspec
import pyqg_parameterization_benchmarks.coarsening_ops as coarsening
from pyqg_parameterization_benchmarks.neural_networks import FCNNParameterization

%matplotlib inline

Although, there are a total of 105 options for constructing FCNN paramaterizations attributed within the paper, we will be solely restricting ourselves to focusing in depth to one parameterization in this guide. We will be training a FCNN on the values of PV to estimate total PV subgrid forcing, computed with Operator 1 (spectral truncation + sharp filter).

We must first load this FCNN paramterization as follows:

In [7]:
param = FCNNParameterization('../models/fcnn_q_to_Sqtot1')

We must also load the coarsened, low-resolution dataset that we will be using as training data for the FCNN parameterization. 

In [8]:
# Initiliaze the model, run it through and filter/coarsen with Operator 1
hires_eddy = pyqg.QGModel(nx=256, dt=3600.0, tmax=311040000.0, tavestart=155520000.0, log_level=0)
hires_eddy.run()
op1 = coarsening.Operator1(hires_eddy, 64) # spectral truncation + sharp filter

INFO:  Logger initialized


In [14]:
op1.m2.to_dataset()

<xarray.Dataset>
Dimensions:  (time: 1, lev: 2, y: 64, x: 64, l: 64, k: 33, lev_mid: 1)
Coordinates:
  * time     (time) float64 0.0
  * lev      (lev) int32 1 2
  * lev_mid  (lev_mid) float64 1.5
  * x        (x) float64 7.812e+03 2.344e+04 3.906e+04 ... 9.766e+05 9.922e+05
  * y        (y) float64 7.812e+03 2.344e+04 3.906e+04 ... 9.766e+05 9.922e+05
  * l        (l) float64 0.0 6.283e-06 1.257e-05 ... -1.257e-05 -6.283e-06
  * k        (k) float64 0.0 6.283e-06 1.257e-05 ... 0.0001948 0.0002011
Data variables: (12/14)
    q        (time, lev, y, x) float64 -1.422e-07 6.696e-06 ... -4.332e-07
    u        (time, lev, y, x) float64 0.1062 0.09177 ... 0.01314 0.01422
    v        (time, lev, y, x) float64 -0.06226 -0.07351 ... 0.006477 0.006766
    ufull    (time, lev, y, x) float64 0.1312 0.1168 0.09474 ... 0.01314 0.01422
    vfull    (time, lev, y, x) float64 -0.06226 -0.07351 ... 0.006477 0.006766
    qh       (time, lev, l, k) complex128 (0.0022066509461801578+0j) ... (-1....
    ...       ...
    ph       (time, lev, l, k) complex128 0j ... (3.3633375281403985e-13-6.22...
    dqhdt    (time, lev, l, k) complex128 0j 0j 0j 0j 0j 0j ... 0j 0j 0j 0j 0j
    Ubg      (lev) float64 0.025 0.0
    Qy       (lev) float64 1.039e-10 -7.222e-12
    dqdt     (time, lev, y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    p        (time, lev, y, x) float64 -576.4 -1.652e+03 ... 462.1 570.3
Attributes: (12/23)
    pyqg:beta:       1.5e-11
    pyqg:delta:      0.25
    pyqg:del2:       0.8
    pyqg:dt:         3600.0
    pyqg:filterfac:  23.6
    pyqg:L:          1000000.0
    ...              ...
    pyqg:tc:         0
    pyqg:tmax:       1576800000.0
    pyqg:twrite:     1000.0
    pyqg:W:          1000000.0
    title:           pyqg: Python Quasigeostrophic Model
    reference:       https://pyqg.readthedocs.io/en/latest/index.html

We can also utilize preexisting datasets as an alternative to our own coarsened data. 

In [ ]:
def get_dataset(path, base_url="https://g-402b74.00888.8540.data.globus.org"):
    mapper = fsspec.get_mapper(f"{base_url}/{path}.zarr")
    return xr.open_zarr(mapper, consolidated=True)

eddy_forcing1 = get_dataset('eddy/forcing1').isel(run=0).load()
eddy_forcing1['q_forcing_total'] = eddy_forcing1.dqdt_bar - eddy_forcing1.dqbar_dt

We can test offline

In [ ]:
preds = param.test_offline(eddy_forcing1)